In [ ]:
import warnings

# Filter out the specific warning
warnings.filterwarnings("ignore", message=".*`PyDataset` class.*")

Loading data

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def get_generator(parent_folder, batch_size=32):
    train_data_dir = os.path.join(parent_folder, 'train')
    validation_data_dir = os.path.join(parent_folder, 'validation')
    test_data_dir = os.path.join(parent_folder, 'test')

    img_width, img_height = 224, 224

    # Define data augmentation parameters
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # For validation and test data, only rescale (no augmentation)
    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Generator for loading and augmenting training data
    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical'  # Assuming you're performing classification
    )

    # Generator for loading validation data
    validation_generator = val_test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical'
    )

    # Generator for loading test data
    test_generator = val_test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical'
    )

    return train_generator, validation_generator, test_generator


# Example usage:
parent_folder = 'F:/thesis/data'
train_generator, validation_generator, test_generator = get_generator(
    parent_folder)

Building model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Define image dimensions
IMAGE_SIZE = (299, 299)  # Xception requires input size of (299, 299)

# Define batch size
BATCH_SIZE = 32

# Define the number of classes
NUM_CLASSES = 38

# Create the Xception model without the top classification layer
base_model = Xception(weights='imagenet', include_top=False,
                      input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# Fine-tune some top layers of the pre-trained model
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Add your own classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)  # Increase number of neurons
x = Dense(512, activation='relu')(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

# Define early stopping callback to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss', patience=5, restore_best_weights=True)

Train model

In [ ]:
# Epoch number:
epochs = 50

# Train the model using the fit method with specified number of epochs
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=epochs,  # Specify the number of epochs here
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate_generator(
    test_generator, steps=test_generator.samples // BATCH_SIZE)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)